In [1]:
from google.colab import drive
# store data in google drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install langchain-openai

In [3]:
import json
import pandas as pd
import os

from pyspark.sql import SparkSession
from pyspark.sql.types import *

from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate


In [4]:
root_path = '/content/drive/My Drive/aig/'
# select AIG and year 2019. The data is in train.jsonl.
sec_file_name = 'train.jsonl'
target_cik = '5272'

data = []
with open(root_path + sec_file_name, encoding="utf-8") as f:
    for row in f:
        data = json.loads(row)
        if data["cik"] == target_cik or int(data["cik"]) == int(target_cik):
            print('found company')
            break

# convert dict into text
text = ""
for k, v in data.items():
    text += k + " : " + v.replace("\n", " ") + ". "


found company


In [5]:
# chunk into pages
# if no page info in the file, approximate the page by number of chars.
def make_chunks(text, chunk_size, chunk_overlap):
    chunks = []
    i = 0
    idx = 0
    while i < len(text):
        chunk = text[i:i+chunk_size]
        chunks.append((idx, chunk))
        i += chunk_size - chunk_overlap
        idx += 1
    return chunks

# num of characters in one page and number of character overlap in two consecutive pages
chunk_size = 2000
chunk_overlap = 200
chunks = make_chunks(text, chunk_size, chunk_overlap)

# in the file, each page ends with a special string "AIG | 2019 Form 10-K"
chunks = text.split('AIG | 2019 Form 10-K')
print(len(chunks))
chunks = [(i, c) for i, c in enumerate(chunks)]
# build spark dataframe
spark = SparkSession.builder.appName("AIG_data_challenge").getOrCreate()
df_chunk = spark.createDataFrame(chunks, schema=StructType([StructField("chunk_id", IntegerType(), False), StructField("chunk_text", StringType(), False)]))

330


In [20]:
from sentence_transformers import SentenceTransformer
import numpy as np
from pyspark.sql.types import *
from pyspark.sql.functions import udf

# use all-MiniLM-L6-v2 as the embedding model to embed each chunk / page
emb_model = SentenceTransformer("all-MiniLM-L6-v2")
embed_udf = udf(lambda x : emb_model.encode(x, normalize_embeddings=True).astype("float32").tolist(), ArrayType(FloatType()))
df_chunk_emb = df_chunk.withColumn("embedding", embed_udf(df_chunk["chunk_text"]))

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [21]:
print(df_chunk)
print(df_chunk.count())
print(df_chunk.head(2))

DataFrame[chunk_id: int, chunk_text: string]
330
[Row(chunk_id=0, chunk_text='filename : 5272_2019.htm. cik : 5272. year : 2019. section_1 : ITEM 1 | Business Regulation BI and Form CRS, and we expect that it will continue to provide guidance regarding these final rules and will further clarify its interpretations through the issuance of additional FAQs and other publications. We continue to evaluate the full impact of this package of final rulemakings and interpretations on us and our customers, distribution partners and financial advisors, while preparing for compliance with them. We will implement and enhance processes and procedures, where needed, to comply with the final rules and interpretations. State Developments The NAIC has been working on revisions to the Suitability in Annuity Transactions Model Regulation (#275), related to the standard of care that would be applicable in a sale or recommendation of an annuity. The proposed revisions were adopted by NAIC’s Life Insurance a

In [22]:
df_chunk_emb.head(2)

[Row(chunk_id=0, chunk_text='filename : 5272_2019.htm. cik : 5272. year : 2019. section_1 : ITEM 1 | Business Regulation BI and Form CRS, and we expect that it will continue to provide guidance regarding these final rules and will further clarify its interpretations through the issuance of additional FAQs and other publications. We continue to evaluate the full impact of this package of final rulemakings and interpretations on us and our customers, distribution partners and financial advisors, while preparing for compliance with them. We will implement and enhance processes and procedures, where needed, to comply with the final rules and interpretations. State Developments The NAIC has been working on revisions to the Suitability in Annuity Transactions Model Regulation (#275), related to the standard of care that would be applicable in a sale or recommendation of an annuity. The proposed revisions were adopted by NAIC’s Life Insurance and Annuities (A) Committee at the end of December

In [23]:
# convert to pandas dataframe for later use
pdf_chunk = df_chunk_emb.toPandas()
c_emb_list = pdf_chunk["embedding"].tolist()

In [25]:
# build rag retriever to retrieve relevant pages
from scipy.spatial.distance import cosine
import numpy as np

def rag_retriever(query, k=1):
    context = []
    q_emb = emb_model.encode([query], normalize_embeddings=True).astype("float32")[0]  # Flatten the query embedding
    for chunk_id in range(len(c_emb_list)):
        c_emb = c_emb_list[chunk_id]
        dist = cosine(np.array(q_emb), np.array(c_emb))
        context.append({
            "similarity": float(1-dist),
            "chunk_id": chunk_id,
            "context": pdf_chunk.loc[pdf_chunk["chunk_id"] == chunk_id]["chunk_text"].values[0]
        })
    # smaller distance means higher similarity
    context.sort(key=lambda x: x['similarity'], reverse=True)
    return context[0:k]

In [27]:
# test rag_retriever
context = rag_retriever("What was AIG total revenues (consolidated) in year 2019? Return a single number in millions.", k=1)
context

[{'similarity': 0.6115107622058166,
  'chunk_id': 305,
  'context': ' ITEM 8 | Notes to Consolidated Financial Statements | 25. Information Provided in Connection with Outstanding Debt Condensed Consolidating Statements of Income (Loss) American International Validus Reclassifications Group, Inc. Holdings, Other and Consolidated (in millions) (As Guarantor) Ltd. AIGLH Subsidiaries Eliminations AIG Year Ended December 31, 2019 Revenues: Equity in earnings of consolidated subsidiaries(a) $ 3,863 $ $ $ - $ (4,491) $ - Other income 1,156 - 48,751 (170) 49,746 Total revenues 5,019 48,751 (4,661) 49,746 Expenses: Interest expense (15) 1,417 Loss on extinguishment of debt - - - - Other expenses 42,415 (155) 43,010 Total expenses 1,714 42,827 (170) 44,459 Income (loss) from continuing operations before income tax expense (benefit) 3,305 5,924 (4,491) 5,287 Income tax expense (benefit) (45) - (6) 1,217 - 1,166 Income (loss) from continuing operations 3,350 4,707 (4,491) 4,121 Income (loss) from

In [45]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from google.colab import userdata

# choose llm model
OPENAI_MODEL = os.environ.get("OPENAI_MODEL", "gpt-4o-mini")
llm = ChatOpenAI(model=OPENAI_MODEL, temperature=0, api_key=userdata.get("openai"))

# construct the contextualized prompt that with retrieved context
def construct_contextualized_prompt(system_prompt_in, context_in, prompt_in):
        system_prompt = (
            "{system_prompt}"
            "\n\n"
            "{context}"
        )
        contextualized_prompt = ChatPromptTemplate.from_messages([
            ("system", system_prompt.format(system_prompt=system_prompt_in, context=context_in)),
            ("human", "{prompt}")
        ])
        return contextualized_prompt.format_messages(prompt=prompt_in)

# llm takes context and prompt to extract data
def extract_data(system_prompt, prompt):
    context = rag_retriever(prompt, k=1)
    contextualized_prompt = construct_contextualized_prompt(system_prompt, context[0]["context"], prompt)
    resp = llm.invoke(contextualized_prompt)
    return {"page_id": context[0]["chunk_id"]+1,
            "page_content": context[0]["context"],
            "similarity": context[0]["similarity"],
            "extraction_result": resp.content.replace("\n", " ")}
#

In [41]:

# prompts
# choose total revenues (consolidated), net income (loss) attributable to AIG as numeric variable, and business operation segments as categorical variable
revenue_prompt = "What was AIG total revenues (consolidated) in year {year}? Return a single number in millions."
income_prompt = "What was AIG net income (loss) attributable to AIG in year {year}? Return a single number in millions."
industry_prompt = "What are AIG's business operation segments? List up to 5. Use exact phrases from the document."

revenue_system_prompt = "Extract the AIG total revenues (consolidated) in year {year} as a single numeric value from the provided context. The answer should be a single numeric value."
income_system_prompt = "Extract the AIG net income (loss) attributable to AIG in year {year} as a single numeric value from the provided context. The answer should be a single numeric value."
industry_system_prompt = "Extract up to 5 distinct business operation segments for AIG from the context. Return the answer as a list."


In [38]:
# test extract_data
extract_data(revenue_system_prompt.replace("{year}", '2019'), revenue_prompt.replace("{year}", '2019'))

{'page_id': 306,
 'page_content': ' ITEM 8 | Notes to Consolidated Financial Statements | 25. Information Provided in Connection with Outstanding Debt Condensed Consolidating Statements of Income (Loss) American International Validus Reclassifications Group, Inc. Holdings, Other and Consolidated (in millions) (As Guarantor) Ltd. AIGLH Subsidiaries Eliminations AIG Year Ended December 31, 2019 Revenues: Equity in earnings of consolidated subsidiaries(a) $ 3,863 $ $ $ - $ (4,491) $ - Other income 1,156 - 48,751 (170) 49,746 Total revenues 5,019 48,751 (4,661) 49,746 Expenses: Interest expense (15) 1,417 Loss on extinguishment of debt - - - - Other expenses 42,415 (155) 43,010 Total expenses 1,714 42,827 (170) 44,459 Income (loss) from continuing operations before income tax expense (benefit) 3,305 5,924 (4,491) 5,287 Income tax expense (benefit) (45) - (6) 1,217 - 1,166 Income (loss) from continuing operations 3,350 4,707 (4,491) 4,121 Income (loss) from discontinued operations, net of i

In [42]:
# ground truth
revenue = {
    '2015': 58327,
    '2016': 52367,
    '2017': 49520,
    '2018': 47389,
    '2019': 49746}

income = {
    '2015': 2196,
    '2016': -849,
    '2017': -6084,
    '2018': -6,
    '2019': 3348}

industry = ['General Insurance', 'Life and Retirement', 'Other Operations', 'Legacy Portfolio', 'North America and International', 'Individual Retirement', 'Group Retirement', 'Life Insurance', 'Institutional Markets',
       'AIG Parent', 'Blackboard', 'Fuji Life', 'Legacy Life and Retirement Run-Off Lines', 'Legacy General Insurance Run-Off Lines', 'Legacy Investments']

In [54]:
# run full pipeline
years = ['2015', '2016', '2017', '2018', '2019']
revenue_results = []
income_results = []
industry_results = []

for year in years:
    res = extract_data(revenue_system_prompt.replace("{year}", year), revenue_prompt.replace("{year}", year))
    res["year"] = year
    res["ground_truth"] = revenue[year]
    revenue_results.append(res)

    res = extract_data(income_system_prompt.replace("{year}", year), income_prompt.replace("{year}", year))
    res["year"] = year
    res["ground_truth"] = income[year]
    income_results.append(res)

res = extract_data(industry_system_prompt, industry_prompt)
res["ground_truth"] = industry
industry_results.append(res)

print(revenue_results)
print(income_results)
print(industry_results)


[{'page_id': 306, 'page_content': ' ITEM 8 | Notes to Consolidated Financial Statements | 25. Information Provided in Connection with Outstanding Debt Condensed Consolidating Statements of Income (Loss) American International Validus Reclassifications Group, Inc. Holdings, Other and Consolidated (in millions) (As Guarantor) Ltd. AIGLH Subsidiaries Eliminations AIG Year Ended December 31, 2019 Revenues: Equity in earnings of consolidated subsidiaries(a) $ 3,863 $ $ $ - $ (4,491) $ - Other income 1,156 - 48,751 (170) 49,746 Total revenues 5,019 48,751 (4,661) 49,746 Expenses: Interest expense (15) 1,417 Loss on extinguishment of debt - - - - Other expenses 42,415 (155) 43,010 Total expenses 1,714 42,827 (170) 44,459 Income (loss) from continuing operations before income tax expense (benefit) 3,305 5,924 (4,491) 5,287 Income tax expense (benefit) (45) - (6) 1,217 - 1,166 Income (loss) from continuing operations 3,350 4,707 (4,491) 4,121 Income (loss) from discontinued operations, net of i

In [56]:
revenue_results

[{'page_id': 306,
  'page_content': ' ITEM 8 | Notes to Consolidated Financial Statements | 25. Information Provided in Connection with Outstanding Debt Condensed Consolidating Statements of Income (Loss) American International Validus Reclassifications Group, Inc. Holdings, Other and Consolidated (in millions) (As Guarantor) Ltd. AIGLH Subsidiaries Eliminations AIG Year Ended December 31, 2019 Revenues: Equity in earnings of consolidated subsidiaries(a) $ 3,863 $ $ $ - $ (4,491) $ - Other income 1,156 - 48,751 (170) 49,746 Total revenues 5,019 48,751 (4,661) 49,746 Expenses: Interest expense (15) 1,417 Loss on extinguishment of debt - - - - Other expenses 42,415 (155) 43,010 Total expenses 1,714 42,827 (170) 44,459 Income (loss) from continuing operations before income tax expense (benefit) 3,305 5,924 (4,491) 5,287 Income tax expense (benefit) (45) - (6) 1,217 - 1,166 Income (loss) from continuing operations 3,350 4,707 (4,491) 4,121 Income (loss) from discontinued operations, net of

In [61]:
# rename columns and save result
df = pd.DataFrame(revenue_results)
df.rename(columns={"extraction_result": "extraction_result(million_USD)",
                   "ground_truth": "ground_truth(millions_USD)"}, inplace=True)
df.to_csv(root_path + "revenue_results.csv", index=False)
df = pd.DataFrame(income_results)
df.rename(columns={"extraction_result": "extraction_result(million_USD)",
                   "ground_truth": "ground_truth(millions_USD)"}, inplace=True)
df.to_csv(root_path + "income_results.csv", index=False)
pd.DataFrame(industry_results).to_csv(root_path + "industry_results.csv", index=False)